In [2]:
!pip install selenium

  Obtaining dependency information for selenium from https://files.pythonhosted.org/packages/e0/7a/08f0ea19a0c835e88aad011083d9dda69a9dfa4585c3453b3bd842eb7bed/selenium-4.21.0-py3-none-any.whl.metadata
  Obtaining dependency information for trio~=0.17 from https://files.pythonhosted.org/packages/76/51/12d78ec8abcbda51d8f115d98ebd3ee3da9d9d9af00ac69d3097c5b8d51a/trio-0.25.1-py3-none-any.whl.metadata
  Obtaining dependency information for trio-websocket~=0.9 from https://files.pythonhosted.org/packages/48/be/a9ae5f50cad5b6f85bd2574c2c923730098530096e170c1ce7452394d7aa/trio_websocket-0.11.1-py3-none-any.whl.metadata
  Obtaining dependency information for typing_extensions>=4.9.0 from https://files.pythonhosted.org/packages/e1/4d/d612de852a0bc64a64418e1cef25fe1914c5b1611e34cc271ed7e36174c8/typing_extensions-4.12.0-py3-none-any.whl.metadata
  Obtaining dependency information for attrs>=23.2.0 from https://files.pythonhosted.org/packages/e0/44/827b2a91a5816512fcaf3cc4ebc465ccd5d598c45cefa670

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydata-profiling 4.6.4 requires visions[type_image_path]==0.7.5, but you have visions 0.7.4 which is incompatible.


In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

from urllib.parse import urlparse, parse_qs, urlencode, urlunparse


import time
import pandas as pd

In [6]:
df = pd.read_csv(r"C:\Users\casti\OneDrive\Documents\A Springboard\Capstone-3---Short-Term-Investments\data\interim\city_urls.csv")
df.head()

,url
0,https://www.airbnb.com/s/newyork-ny/homes
1,https://www.airbnb.com/s/losangeles-ca/homes
2,https://www.airbnb.com/s/chicago-il/homes
3,https://www.airbnb.com/s/dallas-tx/homes
4,https://www.airbnb.com/s/houston-tx/homes


In [7]:
url_list = list(df.url)
url_list[:5]

['https://www.airbnb.com/s/newyork-ny/homes',
 'https://www.airbnb.com/s/losangeles-ca/homes',
 'https://www.airbnb.com/s/chicago-il/homes',
 'https://www.airbnb.com/s/dallas-tx/homes',
 'https://www.airbnb.com/s/houston-tx/homes']

In [8]:
def insert_reviews_path(city_url):
    """Add the string '/reviews' to the appropriate place in an Airbnb listing URL. 
    The new URL navigates to the reviews page of that listing."""
    parsed_url = urlparse(city_url)
    query_params = parse_qs(parsed_url.query)
    new_path = parsed_url.path + "/reviews"
    query_params_str = urlencode(query_params, doseq=True)
    new_url = urlunparse((parsed_url.scheme, parsed_url.netloc, new_path, parsed_url.params, query_params_str, parsed_url.fragment))
    return new_url

def collect_review_urls(city_url, max_pages=5):
    """ 
    Initiates a Selenium webdriver in Chrome that opens an Airbnb URL of listings in a certain city;
    navigates to each listing on the page (20 total) and collects URLs of specific listings;
    uses insert_reviews_path function defined above to transform listing URLs into review page URLs;
    stores and returns these review page URLs as 'review_urls' list.
    """
    listings_class = "itu7ddv"
    driver = webdriver.Chrome()
    driver.get(city_url)

    review_urls = []
    current_page = 1
    
    print(f"Total pages to scan: {max_pages}.")
    
    while current_page <= max_pages:
        WebDriverWait(driver, 60).until(
            EC.visibility_of_all_elements_located((By.CLASS_NAME, listings_class)))
        
        listings = driver.find_elements(By.CLASS_NAME, listings_class)
        
        status = 'Not ready - detecting more listings. . .'
        
        while status == 'Not ready - detecting more listings. . .':
            if len(listings) < 20:
                status = 'Not ready - detecting more listings. . .'
                WebDriverWait(driver, 60).until(
                    EC.visibility_of_all_elements_located((By.CLASS_NAME, listings_class)))
                listings = driver.find_elements(By.CLASS_NAME, listings_class)
                print(status)
                
            else:
                status = f"Ready! {len(listings)} total listings detected on page {current_page}."
                print(status)
                
                if len(listings) > 20:
                    print(f"Estimated {len(listings) - 20} are not valid elements - expect hiccups.")
                    
        for listing in listings:
            try:
                driver.execute_script("arguments[0].scrollIntoView();", listing)
                driver.execute_script("arguments[0].click();", listing)
                
                original_window = driver.current_window_handle
                WebDriverWait(driver, 20).until(EC.number_of_windows_to_be(2))
                handles = driver.window_handles
                driver.switch_to.window(handles[-1])
                
                WebDriverWait(driver, 10).until(EC.url_contains("airbnb.com/rooms/"))
                
                current_url = driver.current_url
                review_url = insert_reviews_path(current_url)
                review_urls.append(review_url)

                driver.close()
                driver.switch_to.window(original_window)
                
            except Exception as e:
                print(f'Could not access listing: {listing}. Exception: {e}')
                continue
        
        # Move to the next page
        if current_page < max_pages:
            try:
                next_page_xpath = "//*[@aria-label='Next']"
                next_button = WebDriverWait(driver, 20).until(
                    EC.element_to_be_clickable((By.XPATH, next_page_xpath)))
                driver.execute_script("arguments[0].scrollIntoView();", next_button)
                next_button.click()
                print(f"Moving to page {current_page + 1} . . .")
                time.sleep(2)
            except Exception as e:
                print(f'Could not navigate to next page: {e}')
                break
        
        current_page += 1
    
    driver.quit()
    return review_urls


In [10]:
def review_scraper(review_url):
    driver = webdriver.Chrome()
    review_data = []
    
    try:
        driver.get(review_url)

        review_class = "r1are2x1"

        WebDriverWait(driver, 60).until(
            EC.visibility_of_all_elements_located((By.CLASS_NAME, review_class)))

        reviews_captured = 0
        max_reviews = 200
        

        while reviews_captured < max_reviews:

            reviews = driver.find_elements(By.CLASS_NAME, review_class)
            current_review_count = len(reviews)

            if current_review_count <= reviews_captured:
                break  # No new reviews are being loaded

            for review in reviews[reviews_captured:]:
                if reviews_captured >= max_reviews:
                    break

                driver.execute_script("arguments[0].scrollIntoView();", review)

                try:
                    # Extracting the relevant data from each review element
                    name = review.find_element(By.CLASS_NAME, "hpipapi").text
                    rating_info = review.find_element(By.CLASS_NAME, "s78n3tv").text
                    review_text = review.find_element(By.CLASS_NAME, "r1bctolv").text

                    # Collecting the data
                    review_data.append({
                        "name": name,
                        "rating_info": rating_info,
                        "review_text": review_text
                    })

                    reviews_captured += 1

                except Exception as e:
                    print(f'Could not extract review data: {e}')
                    continue

            # Scroll down to load more reviews if needed
            if reviews_captured < max_reviews:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                WebDriverWait(driver, 20).until(
                    EC.visibility_of_all_elements_located((By.CLASS_NAME, review_class)))

    except Exception as e:
        print(f"An error occurred while scraping reviews from {review_url}: {e}")
        # Log the error or take other appropriate actions
    
    finally:
        driver.quit()
        
    return review_data


In [11]:
def extract_city_from_url(url):
    # Parse the URL
    parsed_url = urlparse(url)
    
    # Extract the city from the path
    # The city is the first segment after '/s/' in the path
    path_segments = parsed_url.path.split('/')
    
    # Ensure the path structure is as expected
    if len(path_segments) >= 3 and path_segments[1] == 's':
        city = path_segments[2]
        return city
    else:
        raise ValueError("URL structure is not as expected")

In [12]:
def scrapeBNBrevs(city_url):
    
    city_name = extract_city_from_url(city_url)
    
    print(f"Collecting Review URLs for {city_name}. . .")
    
    review_urls = set(collect_review_urls(city_url, max_pages=5))
        
    reviews = []
    max_reviews = 2000
    
    print(f"Capturing reviews for {city_name}. . .")
    for idx, url in enumerate(review_urls):
        
        if len(reviews) >= max_reviews:
            break
        
        print(f"Scanning {idx / 100} potential listings.")
        reviews.extend(review_scraper(url))
        print(f'Updated review count for {city_name}: {len(reviews)} / {max_reviews} max.')

        
    # Filter out duplicate reviews
        
    seen = set()
    unique_reviews = [d for d in reviews if tuple(d.items()) not in seen and not seen.add(tuple(d.items()))]
    print(f'Total unique reviews for {city_name}: {len(unique_reviews)}.')    
    
    review_df = pd.DataFrame(unique_reviews, columns=unique_reviews[0].keys())
    
    return review_df


In [16]:
# Iterate over each city URL
for idx, city_url in enumerate(url_list[-2:]) :

    city_name = extract_city_from_url(city_url)
    
    print('*'*50)
    print(f"{city_name}, {idx} / {len(url_list)} total cities.")
    print('*'*50)
    
    # Scrape reviews for the current city
    city_reviews_df = scrapeBNBrevs(city_url)
    
    # Save the DataFrame with a unique name based on the city name
    filename = r"C:\\Users\\casti\\OneDrive\\Documents\\A Springboard\\Capstone 3 - Investments\\data\\interim\\city_reviews\\{}_reviews.csv".format(city_name)
    city_reviews_df.to_csv(filename, index=False)
    print('*'*50)
    print(f"Saved {city_name} reviews to file.")
    print('*'*50)

**************************************************
vernon-tx, 0 / 894 total cities.
**************************************************
Total pages to scan: 5.
Not ready - detecting more listings. . .
Ready! 20 total listings detected on page 1.
Moving to page 2 . . .
Ready! 22 total listings detected on page 2.
Estimated 2 are not valid elements - expect hiccups.
Could not access listing: <selenium.webdriver.remote.webelement.WebElement (session="ebcb2620027dc1386dbc61c641ec3f25", element="f.0EE7D178BFB53F4C203727167F87FF69.d.2F1ED4FE085C5FA9D80AC6919434714E.e.437")>. Exception: Message: 

Could not access listing: <selenium.webdriver.remote.webelement.WebElement (session="ebcb2620027dc1386dbc61c641ec3f25", element="f.0EE7D178BFB53F4C203727167F87FF69.d.2F1ED4FE085C5FA9D80AC6919434714E.e.440")>. Exception: Message: 

Moving to page 3 . . .
Ready! 28 total listings detected on page 3.
Estimated 8 are not valid elements - expect hiccups.
Could not access listing: <selenium.webdriver.remot

Could not navigate to next page: Message: element click intercepted: Element <button aria-label="Next" type="button" class="l1ovpqvx atm_1he2i46_1k8pnbi_10saat9 atm_yxpdqi_1pv6nv4_10saat9 atm_1a0hdzc_w1h1e8_10saat9 atm_2bu6ew_929bqk_10saat9 atm_12oyo1u_73u7pn_10saat9 atm_fiaz40_1etamxe_10saat9 cz6qhuf atm_1s_glywfm atm_5j_1ssbidh atm_3f_idpfg4 atm_9j_tlke0l atm_tl_1gw4zv3 atm_9s_116y0ak atm_h_1h6ojuz atm_fc_1h6ojuz atm_66_nqa18y atm_6h_t94yts atm_l8_idpfg4 atm_gi_idpfg4 atm_2a_1u8qnfj atm_2d_1kn93ch atm_4b_muzi1o atm_7l_jt7fhx atm_70_1dw9ftv atm_uc_10d7vwn atm_kd_glywfm atm_vy_1vi7ecw atm_e2_1vi7ecw atm_kd_glywfm_1w3cfyq atm_3f_glywfm_e4a3ld atm_l8_idpfg4_e4a3ld atm_gi_idpfg4_e4a3ld atm_3f_glywfm_1r4qscq atm_kd_glywfm_6y7yyg atm_kd_glywfm_pfnrn2_1oszvuo atm_l8_idpfg4_1icshfk_1oszvuo atm_gi_idpfg4_1icshfk_1oszvuo atm_3f_glywfm_b5gff8_1oszvuo atm_kd_glywfm_2by9w9_1oszvuo atm_k4_1piyxwk_1o5j5ji atm_9j_13gfvf7_1o5j5ji atm_uc_glywfm__1rrf6b5 atm_3f_glywfm_jo46a5 atm_l8_idpfg4_jo46a5 atm_gi_

KeyboardInterrupt: 